Fit example using ROOFit

In [ ]:
import pandas as pd
import ROOT as R
from ROOT import RooRealVar, RooGaussian, RooExponential, RooAddPdf, RooArgSet, RooArgList, RooDataSet, RooFit, RooPlot
import matplotlib.pyplot as plt

Import dataset as a RooDataSet

In [ ]:
df = pd.read_csv("../datasets/dataset.csv", header=None)
# convert cvs file into 1D array 
xData = df.values.flatten() 

# determine mass range from data 
xmin, xmax = xData.min(), xData.max()
# define observable
mass = RooRealVar("mass", "Mass [MeV/c^{2}]", xmin, xmax)

# building RooDataSet starting from data
data = RooDataSet("data", "dataset from CSV", RooArgSet(mass))
for val in xData:
    mass.setVal(val)
    data.add(RooArgSet(mass))

Building the fit model with a simple pdf: gaussian for the signal + exponential for the background

In [ ]:
# building fit model 
mean = RooRealVar("mean", "Mean", xData.mean(), 5, 15)
sigma = RooRealVar("sigma", "Sigma", xData.std(), 0., 5.)
gauss = RooGaussian("gauss", "Signal PDF", mass, mean, sigma)

tau = RooRealVar("tau", "Tau", -0.1, -5, 0)  # start from something reasonable
expo = RooExponential("expo", "Background PDF", mass, tau)

nsig = RooRealVar("nsig", "Signal yield", len(xData) / 3, 0, len(xData))
nbkg = RooRealVar("nbkg", "Background yield", len(xData) / 2, 0, len(xData))

model = RooAddPdf("model", "Signal + Background", 
                  RooArgList(gauss, expo), 
                  RooArgList(nsig, nbkg))

Run the fit! and save the results..

In [ ]:
res = model.fitTo(data, RooFit.Extended(True), RooFit.PrintLevel(0), RooFit.Save(True))
res.Print('V')

Let's plot the fit result!

In [ ]:
c1 = R.TCanvas("c1", "Fit to Data", 800, 600)
frame1 = mass.frame()
data.plotOn(frame1)
model.plotOn(
    frame1,
    RooFit.LineColor(R.kRed),
    RooFit.Name('total')
    )
model.plotOn(
    frame1,
    RooFit.Components('gauss'),
    RooFit.LineStyle(R.kDotted),
    RooFit.LineColor(R.kBlue),
    RooFit.Name('gauss')
)
model.plotOn(
    frame1,
    RooFit.Components('expo'),
    RooFit.LineStyle(R.kDotted),
    RooFit.LineColor(R.kOrange),
    RooFit.Name('expo')
)
leg = R.TLegend(0.6, 0.7, 0.88, 0.88)
leg.SetFillStyle(0)
leg.SetBorderSize(0)

total_curve = frame1.findObject('total')
gauss_curve = frame1.findObject('gauss')
expo_curve = frame1.findObject('expo')

leg.AddEntry(gauss_curve, 'Signal', "l")
leg.AddEntry(expo_curve, 'Combinatorial', "l")
leg.AddEntry(total_curve, 'Total', "l")

frame1.SetTitle("Fit to Data from CSV")
frame1.Draw()
leg.Draw()
c1.SaveAs("fit_to_data.pdf")

Plotting pulls distribution

In [ ]:
c1 = R.TCanvas("c1", "Fit to Data", 800, 600)
frame1 = mass.frame()
data.plotOn(frame1)
model.plotOn(
    frame1,
    RooFit.LineColor(R.kRed),
    RooFit.Name('total')
    )
model.plotOn(
    frame1,
    RooFit.Components('gauss'),
    RooFit.LineStyle(R.kDotted),
    RooFit.LineColor(R.kBlue),
    RooFit.Name('gauss')
)
model.plotOn(
    frame1,
    RooFit.Components('expo'),
    RooFit.LineStyle(R.kDotted),
    RooFit.LineColor(R.kOrange),
    RooFit.Name('expo')
)
leg = R.TLegend(0.6, 0.7, 0.88, 0.88)
leg.SetFillStyle(0)
leg.SetBorderSize(0)

total_curve = frame1.findObject('total')
gauss_curve = frame1.findObject('gauss')
expo_curve = frame1.findObject('expo')

leg.AddEntry(gauss_curve, 'Signal', "l")
leg.AddEntry(expo_curve, 'Combinatorial', "l")
leg.AddEntry(total_curve, 'Total', "l")

frame1.SetTitle("Fit to Data from CSV")
frame1.Draw()
leg.Draw()
c1.SaveAs("fit_to_data.pdf")

# TOYS
Once we are happy with the fit model, we can start generating toys, based on the fit results, to check for biases in the parameter estimations. 
You can decide on the number of toy datasets to be generated. 

In [ ]:
n_toy = int(nsig.getVal() + nbkg.getVal())

toy_datasets = []
for i in range(2): 
    toy = model.generate(RooArgSet(mass), n_toy)
    toy_datasets.append(toy)
    # Save it for Systematics_zfit exercise
    # Get all variables in the dataset
    varlist = toy.get()
    columns = [var.GetName() for var in varlist]

    # Extract data
    data = []
    for t in range(toy.numEntries()):
        values = toy.get(t)
        row = [values[var.GetName()].getVal() for var in varlist]
        data.append(row)

    # Save to CSV
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(f"toy_{i}.csv", index=False)
    

Now it's time to fit the toy datasets. 

In [ ]:
toy_results = []
for i, toy in enumerate(toy_datasets):
    print(f"\n--- Fitting toy {i+1} ---")
    result = model.fitTo(toy, Save=True)
    result.Print()
    toy_results.append(result)

What can we do with the toys? 

First, we can check if there is any bias in our fit by looking at the distribution of the fit parameters. 
For instance, we could fit our toy datasets with the same fit model defined above and plot the distribution of the nsig parameter

In [ ]:
fitted_nsig = []
err_nsig = []
for toy in toy_datasets:
    model.fitTo(toy, PrintLevel=-1)
    fitted_nsig.append(nsig.getVal())
    err_nsig.append(nsig.getError())

TODO: 
- Plot histograms of the fitted parameter distributions across the toys. Is the mean value close to the generation one? 
- Increase the number of toys

# Pulls
Pulls are defined as (par_toy - par_ini) / par_toy_unc. They are expected to be centered at 0 with a standard deviation of 1. Also a useful documentation we can provide and they can read about toys: https://hep-physics.rockefeller.edu/luc/technical_reports/cdf5776_pulls.pdf 
To do: plot the pulls of the parameters

In [ ]:
init_par = 2019.4
pulls = []
for i in range(len(fitted_nsig)):
    pulls.append((fitted_nsig[i] - init_par) / err_nsig[i])

Additional tasks: Increase the number of toys, check the other parameters.

# More complicated models: partially-reconstructed background and gaussian constraints

To do:
- load the partreco dataset in a RooDaset
- build a model with the additional part-reco component. 
    The preco model follows a exp(x/xi) * [1 - erf((x - alpha) / beta)] function. You can define a generic pdf with ROOT.RooGenericPdf() (see [documentation](https://root.cern.ch/doc/master/classRooGenericPdf.html)), where you can add directly customised formulas.
    Hint: the erf function can be accessed from TMath::Erf()
- plot the fit results with all the components

In [ ]:
#Here add your solution

<details>
<summary><strong>Click to show the solution</strong></summary>
    
```python
# import dataset with partreco
df = pd.read_csv("../datasets/partreco_dataset1.csv", header=None)
# convert cvs file into 1D array 
xData = df.values.flatten() 

# determine mass range from data 
xmin, xmax = xData.min(), xData.max()
# define observable
mass = RooRealVar("mass", "Mass [MeV/c^{2}]", xmin, xmax)

# building RooDataSet starting from data
data = RooDataSet("data", "dataset from CSV", RooArgSet(mass))
for val in xData:
    mass.setVal(val)
    data.add(RooArgSet(mass))
```
</details>

In [ ]:
# building fit model 
mean = RooRealVar("mean", "Mean", xData.mean(), 5, 15)
sigma = RooRealVar("sigma", "Sigma", xData.std(), 0., 5.)
gauss = RooGaussian("gauss", "Signal PDF", mass, mean, sigma)

tau = RooRealVar("tau", "Tau", -0.1, -5, 0)  # start from something reasonable
expo = RooExponential("expo", "Background PDF", mass, tau)

# part-reco model
# Parameters
alpha = RooRealVar("alpha", "Erf turn-on", 3, 0, 10)
beta  = RooRealVar("beta",  "Erf width", 1, 0.1, 10)
xi    = RooRealVar("xi", "Exponential scale", 5, 0.1, 20)

# Combine exponential and erf via product (this is your part-reco shape)
partReco_expr = "exp(-mass/xi) * (1 - TMath::Erf((mass - alpha)/beta))"
partReco = R.RooGenericPdf("preco", "Partially Reco Bkg", partReco_expr, RooArgList(mass, xi, alpha, beta))


nsig = RooRealVar("nsig", "Signal yield", len(xData) / 3, 0, len(xData))
nbkg = RooRealVar("nbkg", "Background yield", len(xData) / 2, 0, len(xData))
npreco = RooRealVar("npreco", "Partially reconstructed yield", len(xData) / 4, 0, len(xData))

model = RooAddPdf("model", "Signal + Background + Part Reco", 
                  RooArgList(gauss, expo, partReco), 
                  RooArgList(nsig, nbkg, npreco))


# run fit to data 
res = model.fitTo(data, RooFit.Extended(True), RooFit.PrintLevel(0), RooFit.Save(True))

In [ ]:
# plot
c1 = R.TCanvas("c1", "Fit to Data", 800, 600)
frame1 = mass.frame()
data.plotOn(frame1)
model.plotOn(
    frame1,
    RooFit.LineColor(R.kRed),
    RooFit.Name('total')
    )
model.plotOn(
    frame1,
    RooFit.Components('gauss'),
    RooFit.LineStyle(R.kDotted),
    RooFit.LineColor(R.kBlue),
    RooFit.Name('gauss')
)
model.plotOn(
    frame1,
    RooFit.Components('expo'),
    RooFit.LineStyle(R.kDotted),
    RooFit.LineColor(R.kOrange),
    RooFit.Name('expo')
)
model.plotOn(
    frame1,
    RooFit.Components('preco'),
    RooFit.LineStyle(R.kDotted),
    RooFit.LineColor(R.kGreen),
    RooFit.Name('preco')
)

leg = R.TLegend(0.6, 0.7, 0.88, 0.88)
leg.SetFillStyle(0)
leg.SetBorderSize(0)

total_curve = frame1.findObject('total')
gauss_curve = frame1.findObject('gauss')
expo_curve = frame1.findObject('expo')
preco_curve = frame1.findObject('preco')

leg.AddEntry(gauss_curve, 'Signal', "l")
leg.AddEntry(expo_curve, 'Combinatorial', "l")
leg.AddEntry(preco_curve, 'Part reco', "l")
leg.AddEntry(total_curve, 'Total', "l")

# frame1.SetTitle("Fit to Data from CSV")
frame1.Draw()
leg.Draw()
c1.SaveAs("fit_to_data_preco.pdf")

The fit is already quite good, but it might happen that you will have lower statistics, and you will need to fix / constraint some parameters in order to make your fit stable. To fix a parameter p is as simple as setting p.SetConstant(True). But let's see as an example how we can constrain the ratio between signal and part-reco yields, to be around 0.95 (let's say within a 15% uncertainty)

In [ ]:
# Same as before
# import dataset with partreco
df = pd.read_csv("../datasets/partreco_dataset1.csv", header=None)
# convert cvs file into 1D array 
xData = df.values.flatten() 
# determine mass range from data 
xmin, xmax = xData.min(), xData.max()
# define observable
mass = RooRealVar("mass", "Mass [MeV/c^{2}]", xmin, xmax)
# building RooDataSet starting from data
data = RooDataSet("data", "dataset from CSV", RooArgSet(mass))
for val in xData:
    mass.setVal(val)
    data.add(RooArgSet(mass))
# Build model as before
mean = RooRealVar("mean", "Mean", xData.mean(), 5, 15)
sigma = RooRealVar("sigma", "Sigma", xData.std(), 0., 5.)
gauss = RooGaussian("gauss", "Signal PDF", mass, mean, sigma)
tau = RooRealVar("tau", "Tau", -0.1, -5, 0)  # start from something reasonable
expo = RooExponential("expo", "Background PDF", mass, tau)
# part-reco model
# Parameters
alpha = RooRealVar("alpha", "Erf turn-on", 3, 0, 10)
beta  = RooRealVar("beta",  "Erf width", 1, 0.1, 10)
xi    = RooRealVar("xi", "Exponential scale", 5, 0.1, 20)
partReco_expr = "exp(-mass/xi) * (1 - TMath::Erf((mass - alpha)/beta))"
partReco = R.RooGenericPdf("preco", "Partially Reco Bkg", partReco_expr, RooArgList(mass, xi, alpha, beta))

# YIELDS!
nsig = RooRealVar("nsig", "Signal yield", len(xData) / 3, 0, len(xData))
nbkg = RooRealVar("nbkg", "Background yield", len(xData) / 2, 0, len(xData))
#### Here's the main point, as we want to constrain the npreco / nsig quantity. So we will write npreco as
preco_scale = RooRealVar("preco_scale", "Signal over part-reco ratio", 0.9, 0, 1)
npreco = R.RooFormulaVar('npreco', '@0*@1', RooArgList(nsig, preco_scale))
#Define the constraint on preco scale
preco_constr =  RooGaussian("preco_constraint", "preco_constraint", preco_scale, RooFit.RooConst(0.95), RooFit.RooConst(0.20))

model = RooAddPdf("model", "Signal + Background + Part Reco", 
                  RooArgList(gauss, expo, partReco), 
                  RooArgList(nsig, nbkg, npreco),
                  )


In [ ]:
res = model.fitTo(data, RooFit.Extended(True), RooFit.PrintLevel(0), RooFit.Save(True), RooFit.ExternalConstraints(preco_constr)) # Add the constrain)
res.Print('V')

Bonus: if you want you can try to play with the mean / width of the constraint and see how your fit results change.

Bonus: if we were to run toys with this model, how would you expect the pulls of the constrained parameter to look like?

Not a trivial question, if you're curious you can look at: https://arxiv.org/pdf/1210.7141. Main point: the pulls will look exactly as the other parameters, because you need to think about the constraint as an external measurement made on your parameter, which is re-made every time you run a toy. In practical, this means that you need to generate randomly for each toy a new value of the mean of the guassian constraint. 